#Imported Food Information Maru Crawler

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from urllib.request import urlopen
from urllib import parse
from bs4 import BeautifulSoup
import pandas as pd
import datetime

In [3]:
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/bigcontest/data.csv", encoding='utf-8')
squid = data[data['P_NAME'] == '오징어']
selmon = data[data['P_NAME'] == '연어']
shrimp = data[data['P_NAME'] == '흰다리새우']

## URL parsing

https://impfood.mfds.go.kr/CFDAA01F01/getStats? <br>

파라미터들 <br>
page=1<br>
limit=10<br>
active=active<br>
totalCnt=0<br>
hidSrchTon=N<br>
hidSrchKg=Y<br>
statsTypeA=on<br>
statsTypeB=on<br>
statsTypeG=on<br>
statsTypeD=on<br>
statsTypeF=on<br>
srchPrductSeCd=2<br>
srchImpPrposCd=<br>
srchMnfNtncd=<br>
srchXportNtncd=<br>
srchRpsntItmCd=B020602020000002AX00,B0206020200000040000,B0206020200000048500,B0206020200000048529,B0206020200000050000,B0206020200000058300,B0206020200000058500,B020602020000005A400,B020602020000005AC00<br>
srchRpsntItmNm=오징어(냉동,알),오징어(냉장),오징어(냉장,동체),오징어(냉장,동체,자숙),오징어(염장),오징어(염장,다리),오징어(염장,동체),오징어(염장,입살),오징어(염장,지느러미)<br>
srchStrtDate=2018-01-02<br>
srchEndDate=2021-01-01<br>
srchKg=on<br>
<br>
srchRpsntItmCd와 srchRpsntItmNm parameter는 URL parsing이 필요함


In [4]:
# 오징어, 연어, 흰다리새우의 대표품목별 분류
squid_type_lst = squid['P_IMPORT_TYPE'].unique()
squid_type_list = []
squid_types = ''
for i in squid_type_lst:
  squid_types += '오징어('+i+')'+','
  squid_type_list.append('오징어('+i+')')
squid_types = squid_types[:-1]

selmon_type_lst = selmon['P_IMPORT_TYPE'].unique()
selmon_type_list = []
selmon_types = ''
for i in selmon_type_lst:
  selmon_types += '연어('+i+')'+','
  selmon_type_list.append('연어('+i+')')
selmon_types = selmon_types[:-1]

shrimp_type_lst = shrimp['P_IMPORT_TYPE'].unique()
shrimp_type_list = []
shrimp_types = ''
for i in shrimp_type_lst:
  shrimp_types += '흰다리새우('+i+')'+','
  shrimp_type_list.append('흰다리새우('+i+')')
shrimp_types = shrimp_types[:-1]

print(squid_type_lst, selmon_type_lst, shrimp_type_lst)
print(squid_types, selmon_types, shrimp_types)
print(squid_type_list, selmon_type_list, shrimp_type_list)

['냉동,동체' '냉동,다리' '냉동,동체,자숙' '냉동,지느러미' '냉동,슬라이스(S)' '냉동,슬라이스(S),포장횟감' '냉동'
 '냉동,다리,자숙' '냉동,지느러미,자숙' '냉동,난포선' '냉동,동체,포장횟감' '냉동,자숙' '건조'] ['냉장,필렛(F)' '냉장,포장횟감,필렛(F)' '냉장,필렛(F),횟감' '냉장' '냉동' '냉동,곤이'
 '냉동,슬라이스(S),포장횟감' '냉동,필렛(F)'] ['냉동,살,자숙' '냉동,살' '냉동' '냉동,살,자숙,포장횟감' '냉동,살,포장횟감' '냉장' '냉동,자숙']
오징어(냉동,동체),오징어(냉동,다리),오징어(냉동,동체,자숙),오징어(냉동,지느러미),오징어(냉동,슬라이스(S)),오징어(냉동,슬라이스(S),포장횟감),오징어(냉동),오징어(냉동,다리,자숙),오징어(냉동,지느러미,자숙),오징어(냉동,난포선),오징어(냉동,동체,포장횟감),오징어(냉동,자숙),오징어(건조) 연어(냉장,필렛(F)),연어(냉장,포장횟감,필렛(F)),연어(냉장,필렛(F),횟감),연어(냉장),연어(냉동),연어(냉동,곤이),연어(냉동,슬라이스(S),포장횟감),연어(냉동,필렛(F)) 흰다리새우(냉동,살,자숙),흰다리새우(냉동,살),흰다리새우(냉동),흰다리새우(냉동,살,자숙,포장횟감),흰다리새우(냉동,살,포장횟감),흰다리새우(냉장),흰다리새우(냉동,자숙)
['오징어(냉동,동체)', '오징어(냉동,다리)', '오징어(냉동,동체,자숙)', '오징어(냉동,지느러미)', '오징어(냉동,슬라이스(S))', '오징어(냉동,슬라이스(S),포장횟감)', '오징어(냉동)', '오징어(냉동,다리,자숙)', '오징어(냉동,지느러미,자숙)', '오징어(냉동,난포선)', '오징어(냉동,동체,포장횟감)', '오징어(냉동,자숙)', '오징어(건조)'] ['연어(냉장,필렛(F))', '연어(냉장,포장횟감,필렛(F))', '연어(냉장,필렛(F),횟감)', '연어(냉장)', '연어(냉동)', '연어(냉동,곤이)', '연어(냉동,슬라이스(S),포장횟감)', '연어(냉동,필렛(F))'

In [5]:
# 2015-12-28 ~ 2019-12-30 까지 영업일 기준
date_list = list(pd.date_range('20200106', datetime.date(2020,12,28), freq='B'))
date_list = [str(i).split()[0] for i in date_list]
print(date_list)

['2020-01-06', '2020-01-07', '2020-01-08', '2020-01-09', '2020-01-10', '2020-01-13', '2020-01-14', '2020-01-15', '2020-01-16', '2020-01-17', '2020-01-20', '2020-01-21', '2020-01-22', '2020-01-23', '2020-01-24', '2020-01-27', '2020-01-28', '2020-01-29', '2020-01-30', '2020-01-31', '2020-02-03', '2020-02-04', '2020-02-05', '2020-02-06', '2020-02-07', '2020-02-10', '2020-02-11', '2020-02-12', '2020-02-13', '2020-02-14', '2020-02-17', '2020-02-18', '2020-02-19', '2020-02-20', '2020-02-21', '2020-02-24', '2020-02-25', '2020-02-26', '2020-02-27', '2020-02-28', '2020-03-02', '2020-03-03', '2020-03-04', '2020-03-05', '2020-03-06', '2020-03-09', '2020-03-10', '2020-03-11', '2020-03-12', '2020-03-13', '2020-03-16', '2020-03-17', '2020-03-18', '2020-03-19', '2020-03-20', '2020-03-23', '2020-03-24', '2020-03-25', '2020-03-26', '2020-03-27', '2020-03-30', '2020-03-31', '2020-04-01', '2020-04-02', '2020-04-03', '2020-04-06', '2020-04-07', '2020-04-08', '2020-04-09', '2020-04-10', '2020-04-13', '2020

In [6]:
# 각 품종 대표품목별 분류코드
# 대표품목을 url의 koreanNm parameter에 넣은 후 해당 코드 추출
squid_values = ''
for typ in squid_type_list:
  print(typ)
  typ_dict = {'koreanNm': [typ]}
  typ_parse = parse.urlencode(typ_dict, doseq=True)
  url = f'https://impfood.mfds.go.kr/CFECC01P01?page=1&limit=10&callBackFn=fnPopupCallback&usePageId=CFDAA01F01&force=&frmMultiItmCd=&frmMultiItmNm=&tySpcisLclsCd=&itmCd=&useYnNm=&{typ_parse}&btnSearch='
  html = urlopen(url)
  soup = BeautifulSoup(html, 'html.parser')
  select = soup.find_all('tbody')[1]
  
  for i in select.find_all('td'):
    if i.find('input').attrs['alt'] == typ:
      squid_values += i.find('input').attrs['value']+','
      break

squid_values = squid_values[:-1]
print(squid_values)

selmon_values = ''
for typ in selmon_type_list:
  print(typ)
  typ_dict = {'koreanNm': [typ]}
  typ_parse = parse.urlencode(typ_dict, doseq=True)
  url = f'https://impfood.mfds.go.kr/CFECC01P01?page=1&limit=10&callBackFn=fnPopupCallback&usePageId=CFDAA01F01&force=&frmMultiItmCd=&frmMultiItmNm=&tySpcisLclsCd=&itmCd=&useYnNm=&{typ_parse}&btnSearch='
  html = urlopen(url)
  soup = BeautifulSoup(html, 'html.parser')
  select = soup.find_all('tbody')[1]
  
  for i in select.find_all('td'):
    if i.find('input').attrs['alt'] == typ:
      selmon_values += i.find('input').attrs['value']+','
      break

selmon_values = selmon_values[:-1]
print(selmon_values)

shrimp_values = ''
for typ in shrimp_type_list:
  print(typ)
  typ_dict = {'koreanNm': [typ]}
  typ_parse = parse.urlencode(typ_dict, doseq=True)
  url = f'https://impfood.mfds.go.kr/CFECC01P01?page=1&limit=10&callBackFn=fnPopupCallback&usePageId=CFDAA01F01&force=&frmMultiItmCd=&frmMultiItmNm=&tySpcisLclsCd=&itmCd=&useYnNm=&{typ_parse}&btnSearch='
  html = urlopen(url)
  soup = BeautifulSoup(html, 'html.parser')
  select = soup.find_all('tbody')[1]
  
  for i in select.find_all('td'):
    if i.find('input').attrs['alt'] == typ:
      shrimp_values += i.find('input').attrs['value']+','
      break

shrimp_values = shrimp_values[:-1]
print(shrimp_values)

오징어(냉동,동체)
오징어(냉동,다리)
오징어(냉동,동체,자숙)
오징어(냉동,지느러미)
오징어(냉동,슬라이스(S))
오징어(냉동,슬라이스(S),포장횟감)
오징어(냉동)
오징어(냉동,다리,자숙)
오징어(냉동,지느러미,자숙)
오징어(냉동,난포선)
오징어(냉동,동체,포장횟감)
오징어(냉동,자숙)
오징어(건조)
B0206020200000028500,B0206020200000028300,B0206020200000028529,B020602020000002AC00,B0206020200000020023,B0206020200000020024,B0206020200000020000,B0206020200000028329,B020602020000002AC29,B0206020200000027700,B0206020200000028544,B0206020200000020029,B0206020200000010000
연어(냉장,필렛(F))
연어(냉장,포장횟감,필렛(F))
연어(냉장,필렛(F),횟감)
연어(냉장)
연어(냉동)
연어(냉동,곤이)
연어(냉동,슬라이스(S),포장횟감)
연어(냉동,필렛(F))
B0202030000000040048,B0202030000000040045,B0202030000000040050,B0202030000000040000,B0202030000000020000,B0202030000000026900,B0202030000000020024,B0202030000000020048
흰다리새우(냉동,살,자숙)
흰다리새우(냉동,살)
흰다리새우(냉동)
흰다리새우(냉동,살,자숙,포장횟감)
흰다리새우(냉동,살,포장횟감)
흰다리새우(냉장)
흰다리새우(냉동,자숙)
B0205WS0000000020015,B0205WS0000000020019,B0205WS0000000020000,B0205WS0000000020016,B0205WS0000000020021,B0205WS0000000040000,B0205WS0000000020029


## Crawling

In [8]:
rows = []
for date in date_list:
  parse_dict = {"srchRpsntItmCd": squid_values, "srchRpsntItmNm": squid_types}
  parse_parse = parse.urlencode(parse_dict, doseq=True)
  url = f"https://impfood.mfds.go.kr/CFDAA01F01/getStats?page=1&limit=1000&active=active&totalCnt=0&hidSrchTon=N&hidSrchKg=Y&statsTypeA=on&statsTypeB=on&statsTypeG=on&statsTypeD=on&statsTypeF=on&srchPrductSeCd=2&srchImpPrposCd=&srchMnfNtncd=&srchXportNtncd=&{parse_parse}&srchStrtDate={date}&srchEndDate={date}&srchKg=on"
  html = urlopen(url)
  soup = BeautifulSoup(html, 'html.parser')
  select = soup.find('tbody', {'class': 'tb_left'}).find_all('tr')

  if select[0].text == "\n\n\n조회 결과가 없습니다.\n\n":
    continue

  for i in range(len(select)):
    center_lst = select[i].find_all('td', {'class': 'center'})
    center_lst = [j.text for j in center_lst]
    right_lst = select[i].find_all('td', {'class': 'right'})
    right_lst = [j.text for j in right_lst]
    center_lst.extend(right_lst)
    center_lst.insert(0, date)
    rows.append(center_lst)

squid_df = pd.DataFrame(rows, columns=["Date", "제품구분별", "제조국(원산지)별", "수출국별", "수입용도별", "대표품목별", "총계_중량(KG)", "총계_금액($)", "적합_중량(kg)", "적합_금액($)", "부적합_중량(kg)", "부적합_금액($)"])
squid_df.to_csv("/content/drive/MyDrive/Colab Notebooks/bigcontest/squid_val.csv")
squid_df

,Date,제품구분별,제조국(원산지)별,수출국별,수입용도별,대표품목별,총계_중량(KG),총계_금액($),적합_중량(kg),적합_금액($),부적합_중량(kg),부적합_금액($)
0,2020-01-06,수산물,칠레,칠레,판매용,"오징어(냉동,동체)","24,000.000",-,"24,000.000",-,.000,.000
1,2020-01-06,수산물,중국,중국,판매용,"오징어(냉동,슬라이스(S))","1,800.000",-,"1,800.000",-,.000,.000
2,2020-01-06,수산물,페루,페루,판매용,"오징어(냉동,다리)","23,000.000",-,"23,000.000",-,.000,.000
3,2020-01-06,수산물,페루,페루,판매용,"오징어(냉동,지느러미)","22,000.000",-,"22,000.000",-,.000,.000
4,2020-01-06,수산물,중국,중국,판매용,"오징어(냉동,동체)","43,160.000","110,300.000","43,160.000","110,300.000",.000,.000
...,...,...,...,...,...,...,...,...,...,...,...,...
2279,2020-12-28,수산물,중국,중국,판매용,"오징어(냉동,동체)","48,000.000","43,920.000","48,000.000","43,920.000",.000,.000
2280,2020-12-28,수산물,페루,페루,판매용,"오징어(냉동,동체)","22,842.500","34,720.600","22,842.500","34,720.600",.000,.000
2281,2020-12-28,수산물,페루,페루,판매용,"오징어(냉동,지느러미)","154,352.000","205,356.800","154,352.000","205,356.800",.000,.000
2282,2020-12-28,수산물,페루,페루,판매용,"오징어(냉동,동체,자숙)","353,500.000","1,017,440.000","353,500.000","1,017,440.000",.000,.000


In [9]:
rows = []
for date in date_list:
  parse_dict = {"srchRpsntItmCd": selmon_values, "srchRpsntItmNm": selmon_types}
  parse_parse = parse.urlencode(parse_dict, doseq=True)
  url = f"https://impfood.mfds.go.kr/CFDAA01F01/getStats?page=1&limit=1000&active=active&totalCnt=0&hidSrchTon=N&hidSrchKg=Y&statsTypeA=on&statsTypeB=on&statsTypeG=on&statsTypeD=on&statsTypeF=on&srchPrductSeCd=2&srchImpPrposCd=&srchMnfNtncd=&srchXportNtncd=&{parse_parse}&srchStrtDate={date}&srchEndDate={date}&srchKg=on"
  html = urlopen(url)
  soup = BeautifulSoup(html, 'html.parser')
  select = soup.find('tbody', {'class': 'tb_left'}).find_all('tr')

  if select[0].text == "\n\n\n조회 결과가 없습니다.\n\n":
    continue

  for i in range(len(select)):
    center_lst = select[i].find_all('td', {'class': 'center'})
    center_lst = [j.text for j in center_lst]
    right_lst = select[i].find_all('td', {'class': 'right'})
    right_lst = [j.text for j in right_lst]
    center_lst.extend(right_lst)
    center_lst.insert(0, date)
    rows.append(center_lst)

selmon_df = pd.DataFrame(rows, columns=["Date", "제품구분별", "제조국(원산지)별", "수출국별", "수입용도별", "대표품목별", "총계_중량(KG)", "총계_금액($)", "적합_중량(kg)", "적합_금액($)", "부적합_중량(kg)", "부적합_금액($)"])
selmon_df.to_csv("/content/drive/MyDrive/Colab Notebooks/bigcontest/selmon_val.csv")
selmon_df

,Date,제품구분별,제조국(원산지)별,수출국별,수입용도별,대표품목별,총계_중량(KG),총계_금액($),적합_중량(kg),적합_금액($),부적합_중량(kg),부적합_금액($)
0,2020-01-06,수산물,노르웨이,노르웨이,자사제품제조용,연어(냉장),"2,128.300",-,"2,128.300",-,.000,.000
1,2020-01-06,수산물,노르웨이,노르웨이,자사제품제조용,"연어(냉장,필렛(F))","5,996.120","86,735.000","5,996.120","86,735.000",.000,.000
2,2020-01-06,수산물,노르웨이,노르웨이,판매용,"연어(냉장,필렛(F))","11,414.161","187,334.350","11,414.161","187,334.350",.000,.000
3,2020-01-06,수산물,노르웨이,노르웨이,판매용,연어(냉장),"164,789.800","2,224,056.760","164,789.800","2,224,056.760",.000,.000
4,2020-01-07,수산물,캐나다,캐나다,판매용,연어(냉장),861.850,-,861.850,-,.000,.000
...,...,...,...,...,...,...,...,...,...,...,...,...
1235,2020-12-28,수산물,노르웨이,노르웨이,판매용,"연어(냉장,필렛(F))","6,788.900","106,210.000","6,788.900","106,210.000",.000,.000
1236,2020-12-28,수산물,노르웨이,노르웨이,자사제품제조용,연어(냉장),"5,554.400","47,803.860","5,554.400","47,803.860",.000,.000
1237,2020-12-28,수산물,노르웨이,노르웨이,판매용,"연어(냉장,포장횟감,필렛(F))",618.240,"18,527.040",618.240,"18,527.040",.000,.000
1238,2020-12-28,수산물,노르웨이,노르웨이,자사제품제조용,"연어(냉장,필렛(F))","51,063.580","703,810.090","51,063.580","703,810.090",.000,.000


In [10]:
rows = []
for date in date_list:
  parse_dict = {"srchRpsntItmCd": shrimp_values, "srchRpsntItmNm": shrimp_types}
  parse_parse = parse.urlencode(parse_dict, doseq=True)
  url = f"https://impfood.mfds.go.kr/CFDAA01F01/getStats?page=1&limit=1000&active=active&totalCnt=0&hidSrchTon=N&hidSrchKg=Y&statsTypeA=on&statsTypeB=on&statsTypeG=on&statsTypeD=on&statsTypeF=on&srchPrductSeCd=2&srchImpPrposCd=&srchMnfNtncd=&srchXportNtncd=&{parse_parse}&srchStrtDate={date}&srchEndDate={date}&srchKg=on"
  html = urlopen(url)
  soup = BeautifulSoup(html, 'html.parser')
  select = soup.find('tbody', {'class': 'tb_left'}).find_all('tr')

  if select[0].text == "\n\n\n조회 결과가 없습니다.\n\n":
    continue

  for i in range(len(select)):
    center_lst = select[i].find_all('td', {'class': 'center'})
    center_lst = [j.text for j in center_lst]
    right_lst = select[i].find_all('td', {'class': 'right'})
    right_lst = [j.text for j in right_lst]
    center_lst.extend(right_lst)
    center_lst.insert(0, date)
    rows.append(center_lst)

shrimp_df = pd.DataFrame(rows, columns=["Date", "제품구분별", "제조국(원산지)별", "수출국별", "수입용도별", "대표품목별", "총계_중량(KG)", "총계_금액($)", "적합_중량(kg)", "적합_금액($)", "부적합_중량(kg)", "부적합_금액($)"])
shrimp_df.to_csv("/content/drive/MyDrive/Colab Notebooks/bigcontest/shrimp_val.csv")
shrimp_df

,Date,제품구분별,제조국(원산지)별,수출국별,수입용도별,대표품목별,총계_중량(KG),총계_금액($),적합_중량(kg),적합_금액($),부적합_중량(kg),부적합_금액($)
0,2020-01-06,수산물,인도,인도,판매용,"흰다리새우(냉동,살)","18,000.000",-,"18,000.000",-,.000,.000
1,2020-01-06,수산물,태국,태국,판매용,"흰다리새우(냉동,자숙)","6,200.000",-,"6,200.000",-,.000,.000
2,2020-01-06,수산물,태국,태국,판매용,흰다리새우(냉장),320.000,-,320.000,-,.000,.000
3,2020-01-06,수산물,베트남,베트남,판매용,"흰다리새우(냉동,자숙)","3,150.000",-,"3,150.000",-,.000,.000
4,2020-01-06,수산물,베트남,베트남,판매용,"흰다리새우(냉동,살,포장횟감)","9,102.000","92,704.400","9,102.000","92,704.400",.000,.000
...,...,...,...,...,...,...,...,...,...,...,...,...
1599,2020-12-28,수산물,페루,페루,판매용,흰다리새우(냉동),"63,000.000","397,473.600","63,000.000","397,473.600",.000,.000
1600,2020-12-28,수산물,태국,태국,판매용,"흰다리새우(냉동,살,자숙,포장횟감)","10,606.000","218,998.000","10,606.000","218,998.000",.000,.000
1601,2020-12-28,수산물,베트남,베트남,판매용,"흰다리새우(냉동,살,자숙,포장횟감)","14,153.000","235,753.920","14,153.000","235,753.920",.000,.000
1602,2020-12-28,수산물,태국,태국,판매용,"흰다리새우(냉동,살)","43,697.700","396,033.290","43,697.700","396,033.290",.000,.000
